<a href="https://colab.research.google.com/github/drscghosh/Testing/blob/master/DiallelGriffingMethod2Model1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
from itertools import combinations_with_replacement

# Generate a sample dataset: Griffing Method 2 (parents + one-way crosses)
def generate_sample_data():
    parents = ['P1', 'P2', 'P3', 'P4']
    crosses = list(combinations_with_replacement(parents, 2))
    data = []
    np.random.seed(0)
    for i, (p1, p2) in enumerate(crosses):
        yield_val = np.round(np.random.normal(loc=50, scale=5), 2)
        data.append([p1, p2, yield_val])
    return pd.DataFrame(data, columns=["Parent1", "Parent2", "Yield"])

# Create the sample dataset
df = generate_sample_data()
print("Sample Diallel Data (Method 2):")
print(df)

# Create a unique cross identifier
df["Cross"] = df[["Parent1", "Parent2"]].apply(lambda x: "-".join(sorted([x[0], x[1]])), axis=1)

# ANOVA using OLS with fixed effects
model = ols('Yield ~ C(Parent1) + C(Parent2)', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print("\nANOVA Table:")
print(anova_table)

# Compute GCA and SCA
parents = sorted(df['Parent1'].unique())
n_parents = len(parents)

# Mean yield of each parent (including its crosses)
gca = {}
overall_mean = df["Yield"].mean()
parent_means = {p: df[(df["Parent1"] == p) | (df["Parent2"] == p)]["Yield"].mean() for p in parents}

# GCA = mean of crosses involving the parent - overall mean
gca = {p: round(parent_means[p] - overall_mean, 2) for p in parents}

print("\nGeneral Combining Ability (GCA):")
for p in gca:
    print(f"{p}: {gca[p]}")

# SCA = specific cross mean - overall mean - GCA of P1 - GCA of P2
print("\nSpecific Combining Ability (SCA):")
scas = []
for _, row in df.iterrows():
    p1, p2 = row["Parent1"], row["Parent2"]
    cross_mean = row["Yield"]
    sca = round(cross_mean - overall_mean - gca[p1] - gca[p2], 2)
    scas.append((f"{p1}-{p2}", sca))
    print(f"{p1}-{p2}: {sca}")



Sample Diallel Data (Method 2):
  Parent1 Parent2  Yield
0      P1      P1  58.82
1      P1      P2  52.00
2      P1      P3  54.89
3      P1      P4  61.20
4      P2      P2  59.34
5      P2      P3  45.11
6      P2      P4  54.75
7      P3      P3  49.24
8      P3      P4  49.48
9      P4      P4  52.05

ANOVA Table:
                sum_sq   df         F    PR(>F)
C(Parent1)   48.900225  3.0  0.477846  0.720159
C(Parent2)   53.298500  3.0  0.520826  0.697213
Residual    102.334642  3.0       NaN       NaN

General Combining Ability (GCA):
P1: 3.04
P2: -0.89
P3: -4.01
P4: 0.68

Specific Combining Ability (SCA):
P1-P1: -0.95
P1-P2: -3.84
P1-P3: 2.17
P1-P4: 3.79
P2-P2: 7.43
P2-P3: -3.68
P2-P4: 1.27
P3-P3: 3.57
P3-P4: -0.88
P4-P4: -3.0


<ipython-input-1-98d673f9cdd4>:24: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df["Cross"] = df[["Parent1", "Parent2"]].apply(lambda x: "-".join(sorted([x[0], x[1]])), axis=1)
